In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [56]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
}

In [57]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [58]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,100,67.77,51.25
1,H-UNet-SWTA-T,0,5,100,58.24,41.08
2,H-UNet-SWTA-T,0,10,100,53.94,36.93
3,H-UNet-SWTA-T,0,20,100,63.43,46.44
4,H-UNet-SWTA-T,0,50,100,66.33,49.62
5,H-UNet-SWTA-T,0,100,100,64.64,47.75


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                Mean CI 0.9%    Mean CI 0.9%
Model         Inv Temp Regime                               
H-UNet-SWTA-T 1        100     67.77     NaN   51.25     NaN
              5        100     58.24     NaN   41.08     NaN
              10       100     53.94     NaN   36.93     NaN
              20       100     63.43     NaN   46.44     NaN
              50       100     66.33     NaN   49.62     NaN
              100      100     64.64     NaN   47.75     NaN

In [10]:
EXP_ROOT = "./runs_bis"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
}

In [11]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [12]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,1,67.78,51.26
1,H-UNet-SWTA-T,0,1,2,67.80,51.28
2,H-UNet-SWTA-T,0,1,5,68.00,51.52
3,H-UNet-SWTA-T,0,1,10,67.98,51.49
4,H-UNet-SWTA-T,0,1,20,70.14,54.01
...,...,...,...,...,...,...
295,H-UNet-SWTA-T,9,100,1,67.77,51.25
296,H-UNet-SWTA-T,9,100,2,67.77,51.25
297,H-UNet-SWTA-T,9,100,5,70.10,53.96
298,H-UNet-SWTA-T,9,100,10,74.21,59.00


Dice           Jaccard          
                                 Mean   CI 0.9%    Mean   CI 0.9%
Model         Inv Temp Regime                                    
H-UNet-SWTA-T 1        1       67.822  0.067193  51.310  0.075936
                       2       67.638  0.266777  51.101  0.300345
                       5       67.928  0.062099  51.432  0.071927
                       10      67.874  0.062283  51.370  0.072170
                       20      68.419  0.449568  51.999  0.522443
              5        1       67.953  0.095676  51.460  0.110839
                       2       68.149  0.117074  51.686  0.132167
                       5       70.337  1.046533  54.275  1.243024
                       10      72.833  1.218558  57.316  1.490271
                       20      75.286  1.051658  60.399  1.339403
              10       1       67.957  0.167608  51.465  0.192943
                       2       67.895  0.130583  51.393  0.149554
                       5       69.121  0.570286  52.820  0.668953
                       10      70.509  1.028013  54.478  1.224834
                       20      75.449  1.075291  60.611  1.378860
              20       1       68.049  0.180549  51.572  0.207972
                       2       68.382  0.690511  51.968  0.817420
                       5       69.789  0.884402  53.617  1.049659
                       10      73.587  1.207505  58.255  1.516777
                       20      76.773  0.522441  62.308  0.690399
              50       1       68.252  0.233573  51.806  0.268827
                       2       68.911  0.840356  52.589  0.995961
                       5       72.094  0.897863  56.387  1.091609
                       10      75.102  0.696765  60.145  0.893432
                       20      78.871  0.862606  65.138  1.169342
              100      1       67.794  0.038030  51.277  0.043228
                       2       68.062  0.272645  51.590  0.316890
                       5       70.495  0.823679  54.452  0.980860
                       10      74.152  0.789090  58.938  0.987545
                       20      78.158  0.948323  64.177  1.273060

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [53]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    #'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    #'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
    #'CPS': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cps_unet')),
    'URPC': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('urpc_unet')),
}

In [54]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [55]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,URPC,0,1,1,68.03,51.55
1,URPC,0,1,2,67.84,51.33
2,URPC,0,1,5,72.85,57.30
3,URPC,0,1,10,76.63,62.11
4,URPC,0,1,20,81.68,69.03
5,URPC,1,1,1,67.85,51.35
6,URPC,1,1,2,68.23,51.78
7,URPC,1,1,5,72.64,57.04
8,URPC,1,1,10,81.41,68.64
9,URPC,1,1,20,85.75,75.05


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
URPC  1        1       69.214  0.964375  52.948  1.138827
               2       69.169  1.218885  52.909  1.460061
               5       74.254  1.527695  59.119  1.918430
               10      77.626  1.610150  63.515  2.136674
               20      81.130  2.366412  68.440  3.286478